In [1]:
import os
import pickle
import argparse
import unicodedata
import pandas as pd

# Import Tokenizer
from transformers import AutoTokenizer
from kobert_tokenizer import KoBERTTokenizer

In [2]:
def define_argparser():
    p = argparse.ArgumentParser()

    p.add_argument("--pretrained_model_name", default ='klue/roberta-base', help="Pre-trained model name to be going to train. Tokenizer will be assigned based on the model.")
    p.add_argument("--load_fn", default='data/dataset/train.pickle', help="Original data to be going to preprocess.")
    p.add_argument("--save_path", default='data/encoded', help="Original data to be going to preprocess.")
    p.add_argument("--with_text", action="store_true", help="Return values with text for inference.")

    config = p.parse_args(args=[])

    return config

In [3]:
def get_char_labels(sentence, ne, label_to_index, normalize=False):
    """
    Labeling tags by character-level.
    Parameter "normalize" for KoBERTTokenizer, which used to normalize unicode text before tokenizing.
    """
    char_labels = [0] * len(sentence)
    for i in range(1, len(ne)+1):
        begin = ne[i]['begin']
        end = ne[i]['end']
        B_label = 'B-'+ne[i]['label'][:2]
        I_label = 'I-'+ne[i]['label'][:2]
        for j in range(begin, end):
            char_labels[j] = label_to_index[I_label]
        char_labels[begin] = label_to_index[B_label]

    if normalize:
        normalized_sentence = unicodedata.normalize("NFKC", sentence)
        for i, char in enumerate(sentence):
            normalized_char = unicodedata.normalize("NFKC", char)
            if len(normalized_char) > 1:
                char_labels[i:i] = [char_labels[i]] * (len(normalized_char)-1)
                if len(char_labels) == len(normalized_sentence):
                    break        

    return char_labels


def get_kobert_offset_mappings(kobert_tokens):
    """
    This function is for KoBERTTokenizer, which cannot return offset_mappings.
    """
    kobert_offset_mappings = []
    offset_begin = -1
    for i in kobert_tokens:
        if i in ["[CLS]", "[SEP]"]:
            kobert_offset_mappings.append((0, 0))
            continue
        if i.startswith('▁'):
            offset_begin += 1
            i = i[1:]
        offset_end = offset_begin+len(i)
        kobert_offset_mappings.append((offset_begin, offset_end))
        offset_begin = offset_end

    return kobert_offset_mappings


def get_token_labels(char_labels, offset_mappings):
    """
    Labeling tags by token-level.
    """
    token_labels = []

    for offset_mapping in offset_mappings:
        start_offset, end_offset = offset_mapping
        if start_offset == end_offset:
            token_labels.append(-100)
            continue
        token_labels.append(char_labels[start_offset])

    return token_labels


def get_label_dict(labels):
    BIO_labels = ['O']
    for label in labels:
        BIO_labels.append('B-'+label)
        BIO_labels.append('I-'+label)

    label_to_index = {label: index for index, label in enumerate(BIO_labels)}
    index_to_label = {index: label for index, label in enumerate(BIO_labels)}

    return label_to_index, index_to_label

In [4]:
config = define_argparser()

In [5]:
USE_KOBERT = True if config.pretrained_model_name == 'skt/kobert-base-v1' else False
data = pd.read_pickle(config.load_fn)
texts = data['sentence'].values.tolist()
nes = data['ne'].values.tolist()

In [6]:
data

,sentence,ne,source,sentence_class
213668,오늘 왜 머리 올리고 왔어?,"{1: {'form': '오늘', 'label': 'DT_DAY', 'begin':...",S,AM
24325,이에 대해 문화재청 관계자는 “조사시기는 조사전문기관이 자체 판단해 결정한 것”이라...,"{1: {'form': '문화재청', 'label': 'OGG_POLITICS', ...",N,OG
64711,잘나가는 ‘상사맨’으로 일했던 장 대표는 2009년 새로운 도전을 시작했다.,"{1: {'form': '장', 'label': 'PS_NAME', 'begin':...",N,DT
56810,국방부는 또 회담 전날까지 공동 보도문에 ‘한반도의 평화와 안정을 해치는 어떠한 도...,"{1: {'form': '국방부', 'label': 'OGG_POLITICS', '...",N,LC
297764,아니에요 팔백이에요,"{1: {'form': '팔백', 'label': 'QT_PRICE', 'begin...",S,QT
...,...,...,...,...
162864,또 우리가 공항에 일 등 공항 이라는,"{1: {'form': '일 등', 'label': 'QT_ORDER', 'begi...",S,QT
121479,하지만 쿠웨이트로 떠나기도 전에 주축 선수들이 부상을 당하며 슈틸리케호(號)엔 먹구...,"{1: {'form': '쿠웨이트', 'label': 'LCP_COUNTRY', '...",N,LC
3474,"현재 심사 중인 비상임위원의 임명이 끝나면, 위원들이 위원장 후보 5명 중 한 명을...","{1: {'form': '위원', 'label': 'CV_POSITION', 'be...",N,QT
329335,아니. 보세요. 산부인과 의사한테 진료를 받으면서 한 번이라도 추행을 느껴본 적 있어요?,"{1: {'form': '산부인과 의사', 'label': 'CV_OCCUPATIO...",S,QT


In [7]:
texts

['오늘 왜 머리 올리고 왔어?',
 '이에 대해 문화재청 관계자는 “조사시기는 조사전문기관이 자체 판단해 결정한 것”이라며 “표본시굴조사를 통해 집터 등 유물이 확인되면 전면조사를 할 예정”이라고 해명했다.',
 '잘나가는 ‘상사맨’으로 일했던 장 대표는 2009년 새로운 도전을 시작했다.',
 '국방부는 또 회담 전날까지 공동 보도문에 ‘한반도의 평화와 안정을 해치는 어떠한 도발행위에도 반대한다’는 문구를 포함시키기로 중국과 협의했지만 회담 당일 중국이 ‘도발’ 용어를 뺄 것을 강력히 요구해 결국 이를 수용했다.',
 '아니에요 팔백이에요',
 '단적으로 말해, “개발독재의 방식으로 신자유주의를 추진하는” 성격을 갖는다는 풀이다.',
 "근현대 서양미술 명작 114점을 한자리에서 볼 수 있는 '모네에서 워홀까지' 특별전이 관람객 10만명을 돌파했다.",
 '돈은 마련했지만 사업자 등록은 동생 이름으로 했다.',
 '우리 신랑도 맨날 오면 그래',
 '○ 현대차, 세계적인 대회에 재도전',
 '게이센여학원대 국제사회학과',
 '그 네 롤 중에서 한 롤밖에 스캔 안 했어.',
 '피아노는 폭풍처럼 몰아쳤다가 나뭇잎처럼 살랑댔고 한순간 따사로운 봄날이었다가는 부르르 떨며 서글픈 현실을 상기시켰다.',
 '조금이라도 발을 빼면 역적이라는 분위기가 보수진영에 있다.',
 '보합세라는 응답이 11명(55%)으로 가장 많았고, 1∼2%대 오를 것이라는 응답이 7명(35%)으로 뒤를 이었다.',
 '이들은 한결같이 "근데 박준우사진>가 도대체 누구야"라고 서로에게 물었다.',
 '현재 국내 등록 차량 1679만대 가운데 2000년 이전 등록 차는 548만대다.',
 '친박계의 한 최고위원은 “유 전 원내대표가 유치원생도 아닌데 공천을 안 준다는 상황에서 출마할 수 있겠느냐.',
 '그래서 다른 나라 필리핀도 가보면 뭐',
 '또한 그~ 철도를 작년 연말에',
 "전남대 컨벤션센터 용지홀에서는 이날 오후 2시 '1991년 청춘의 기억'이라는 주제로 전시회가 개막됐다.",


In [8]:
nes

[{1: {'form': '오늘', 'label': 'DT_DAY', 'begin': 0, 'end': 2},
  2: {'form': '머리', 'label': 'AM_PART', 'begin': 5, 'end': 7}},
 {1: {'form': '문화재청', 'label': 'OGG_POLITICS', 'begin': 6, 'end': 10}},
 {1: {'form': '장', 'label': 'PS_NAME', 'begin': 17, 'end': 18},
  2: {'form': '대표', 'label': 'CV_POSITION', 'begin': 19, 'end': 21},
  3: {'form': '2009년', 'label': 'DT_YEAR', 'begin': 23, 'end': 28}},
 {1: {'form': '국방부', 'label': 'OGG_POLITICS', 'begin': 0, 'end': 3},
  2: {'form': '전날까지', 'label': 'DT_OTHERS', 'begin': 10, 'end': 14},
  3: {'form': '한반도', 'label': 'LCG_BAY', 'begin': 24, 'end': 27},
  4: {'form': '중국', 'label': 'OGG_POLITICS', 'begin': 70, 'end': 72},
  5: {'form': '중국', 'label': 'OGG_POLITICS', 'begin': 86, 'end': 88}},
 {1: {'form': '팔백', 'label': 'QT_PRICE', 'begin': 5, 'end': 7}},
 {1: {'form': '신자유주의', 'label': 'TR_SOCIAL_SCIENCE', 'begin': 21, 'end': 26}},
 {1: {'form': '근현대 서양미술', 'label': 'CV_ART', 'begin': 0, 'end': 8},
  2: {'form': '114점', 'label': 'QT_COUNT', 

In [9]:
pretrained_model_name = config.pretrained_model_name
print(pretrained_model_name)

klue/roberta-base


In [10]:
if USE_KOBERT:
    tokenizer_loader = KoBERTTokenizer # 해당 안 됨
else:
    tokenizer_loader = AutoTokenizer

tokenizer = tokenizer_loader.from_pretrained(pretrained_model_name)
print("Tokenizer loaded :", tokenizer.name_or_path)

Tokenizer loaded : klue/roberta-base


In [11]:
label_list = ["PS", "FD", "TR", "AF", "OG", "LC", "CV", "DT", "TI", "QT", "EV", "AM", "PT", "MT", "TM"]
label_to_index, index_to_label = get_label_dict(label_list) # 레이블 리스트를 받아서 BIO 매핑을 진행
print(label_to_index)
print(index_to_label)
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
max_length = 512

{'O': 0, 'B-PS': 1, 'I-PS': 2, 'B-FD': 3, 'I-FD': 4, 'B-TR': 5, 'I-TR': 6, 'B-AF': 7, 'I-AF': 8, 'B-OG': 9, 'I-OG': 10, 'B-LC': 11, 'I-LC': 12, 'B-CV': 13, 'I-CV': 14, 'B-DT': 15, 'I-DT': 16, 'B-TI': 17, 'I-TI': 18, 'B-QT': 19, 'I-QT': 20, 'B-EV': 21, 'I-EV': 22, 'B-AM': 23, 'I-AM': 24, 'B-PT': 25, 'I-PT': 26, 'B-MT': 27, 'I-MT': 28, 'B-TM': 29, 'I-TM': 30}
{0: 'O', 1: 'B-PS', 2: 'I-PS', 3: 'B-FD', 4: 'I-FD', 5: 'B-TR', 6: 'I-TR', 7: 'B-AF', 8: 'I-AF', 9: 'B-OG', 10: 'I-OG', 11: 'B-LC', 12: 'I-LC', 13: 'B-CV', 14: 'I-CV', 15: 'B-DT', 16: 'I-DT', 17: 'B-TI', 18: 'I-TI', 19: 'B-QT', 20: 'I-QT', 21: 'B-EV', 22: 'I-EV', 23: 'B-AM', 24: 'I-AM', 25: 'B-PT', 26: 'I-PT', 27: 'B-MT', 28: 'I-MT', 29: 'B-TM', 30: 'I-TM'}


In [12]:
if USE_KOBERT:
    encoded = tokenizer(texts,
                        add_special_tokens=True,
                        padding=False,
                        return_attention_mask=True,
                        truncation=True,
                        max_length=max_length)
else:
    encoded = tokenizer(texts,
                        add_special_tokens=True,
                        padding=False,
                        return_attention_mask=True,
                        truncation=True,
                        max_length=max_length,
                        return_offsets_mapping=True)

print("Sentences encoded : |input_ids| %d, |attention_mask| %d" %
        (len(encoded['input_ids']), len((encoded['attention_mask']))))

Sentences encoded : |input_ids| 166145, |attention_mask| 166145


In [13]:
df = pd.DataFrame([encoded["input_ids"], encoded["attention_mask"], data['sentence_class'].values],
                  index=["input_ids", "attention_mask", "sentence_class"]).T

In [14]:
df

,input_ids,attention_mask,sentence_class
0,"[0, 3822, 1460, 4100, 4705, 2088, 1458, 2051, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",AM
1,"[0, 1504, 2170, 3643, 16502, 3810, 2259, 116, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",OG
2,"[0, 1521, 16570, 2259, 114, 7326, 2763, 115, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",DT
3,"[0, 7006, 2259, 918, 4823, 5978, 2299, 2118, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",LC
4,"[0, 3614, 2170, 2182, 1830, 2353, 2052, 2170, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",QT
...,...,...,...
166140,"[0, 918, 3616, 2116, 4771, 2170, 1507, 886, 47...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",QT
166141,"[0, 3696, 20821, 2200, 4614, 2015, 2119, 1537,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",LC
166142,"[0, 3738, 4917, 1570, 2179, 5506, 2289, 8956, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",QT
166143,"[0, 3614, 18, 31153, 2182, 18, 17742, 4206, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",QT


In [15]:
ne_ids = []

if USE_KOBERT:
    text_tokens = [[cls_token] + tokenizer.tokenize(text)[:max_length-2] + [sep_token] for text in texts]
    offset_mappings = [get_kobert_offset_mappings(text_token) for text_token in text_tokens]
    text_normalize=True
else: 
    offset_mappings = encoded['offset_mapping']
    text_normalize=False

In [16]:
encoded['offset_mapping'][0]

[(0, 0),
 (0, 2),
 (3, 4),
 (5, 7),
 (8, 10),
 (10, 11),
 (12, 13),
 (13, 14),
 (14, 15),
 (0, 0)]

In [17]:
for text, ne, offset_mapping in zip(texts, nes, offset_mappings):
    char_labels = get_char_labels(text, ne, label_to_index, normalize=text_normalize)
    token_labels = get_token_labels(char_labels, offset_mapping)
    ne_ids.append(token_labels)

print("Sequence labeling completed : |labels| %d" % (len(ne_ids)))

Sequence labeling completed : |labels| 166145


In [18]:
return_data = pd.DataFrame([encoded["input_ids"], encoded["attention_mask"], ne_ids, data['sentence_class'].values], index=[
                            "input_ids", "attention_mask", "labels", "sentence_class"]).T
if config.with_text:
    return_data['sentence'] = texts

In [19]:
return_data

,input_ids,attention_mask,labels,sentence_class
0,"[0, 3822, 1460, 4100, 4705, 2088, 1458, 2051, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 15, 0, 23, 0, 0, 0, 0, 0, -100]",AM
1,"[0, 1504, 2170, 3643, 16502, 3810, 2259, 116, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",OG
2,"[0, 1521, 16570, 2259, 114, 7326, 2763, 115, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 13,...",DT
3,"[0, 7006, 2259, 918, 4823, 5978, 2299, 2118, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 9, 0, 0, 0, 15, 16, 16, 0, 0, 0, 0, 0, ...",LC
4,"[0, 3614, 2170, 2182, 1830, 2353, 2052, 2170, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 0, 0, 0, 19, 20, 0, 0, 0, -100]",QT
...,...,...,...,...
166140,"[0, 918, 3616, 2116, 4771, 2170, 1507, 886, 47...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 0, 0, 0, 0, 0, 19, 20, 0, 0, 0, -100]",QT
166141,"[0, 3696, 20821, 2200, 4614, 2015, 2119, 1537,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 11, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0...",LC
166142,"[0, 3738, 4917, 1570, 2179, 5506, 2289, 8956, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",QT
166143,"[0, 3614, 18, 31153, 2182, 18, 17742, 4206, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 13, 14, 0, 0, 0, 0, 0, 0...",QT


In [21]:
label_info = {
    "label_list": label_list,
    "label_to_index": label_to_index,
    "index_to_label": index_to_label
}
print(label_info)

{'label_list': ['PS', 'FD', 'TR', 'AF', 'OG', 'LC', 'CV', 'DT', 'TI', 'QT', 'EV', 'AM', 'PT', 'MT', 'TM'], 'label_to_index': {'O': 0, 'B-PS': 1, 'I-PS': 2, 'B-FD': 3, 'I-FD': 4, 'B-TR': 5, 'I-TR': 6, 'B-AF': 7, 'I-AF': 8, 'B-OG': 9, 'I-OG': 10, 'B-LC': 11, 'I-LC': 12, 'B-CV': 13, 'I-CV': 14, 'B-DT': 15, 'I-DT': 16, 'B-TI': 17, 'I-TI': 18, 'B-QT': 19, 'I-QT': 20, 'B-EV': 21, 'I-EV': 22, 'B-AM': 23, 'I-AM': 24, 'B-PT': 25, 'I-PT': 26, 'B-MT': 27, 'I-MT': 28, 'B-TM': 29, 'I-TM': 30}, 'index_to_label': {0: 'O', 1: 'B-PS', 2: 'I-PS', 3: 'B-FD', 4: 'I-FD', 5: 'B-TR', 6: 'I-TR', 7: 'B-AF', 8: 'I-AF', 9: 'B-OG', 10: 'I-OG', 11: 'B-LC', 12: 'I-LC', 13: 'B-CV', 14: 'I-CV', 15: 'B-DT', 16: 'I-DT', 17: 'B-TI', 18: 'I-TI', 19: 'B-QT', 20: 'I-QT', 21: 'B-EV', 22: 'I-EV', 23: 'B-AM', 24: 'I-AM', 25: 'B-PT', 26: 'I-PT', 27: 'B-MT', 28: 'I-MT', 29: 'B-TM', 30: 'I-TM'}}


In [22]:
return_values = {
    "data": return_data.to_dict(),
    "label_info": label_info,
    "pad_token": (tokenizer.pad_token, tokenizer.pad_token_id),
    "pretrained_model_name": tokenizer.name_or_path
}

In [23]:
return_values['pad_token']

('[PAD]', 1)

In [24]:
return_values['pretrained_model_name']

'klue/roberta-base'

In [25]:
save_path = config.save_path   
fn = os.path.split(config.load_fn)[1].split('.')[0]
plm_name = pretrained_model_name.replace('/', '_')
save_fn = os.path.join(save_path, f'{fn}.{plm_name}.encoded.pickle')

In [26]:
print(save_path)
print(fn)
print(plm_name)
print(save_fn)

data/encoded
train
klue_roberta-base
data/encoded/train.klue_roberta-base.encoded.pickle


In [20]:
# with open(save_fn, "wb") as f:
#     pickle.dump(return_values, f, 4)
# print("Encoded data saved as %s " % save_fn)